In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
pwd

#### Import MNIST

In [ ]:
import numpy as np
import os
import struct

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

def load_mnist(image_path, label_path):
    images = read_idx(image_path)
    labels = read_idx(label_path)
    return images, labels

train_image_path = './drive/MyDrive/MNIST/train-images-idx3-ubyte/train-images-idx3-ubyte'
train_label_path = './drive/MyDrive/MNIST/train-labels-idx1-ubyte/train-labels-idx1-ubyte'
test_image_path =  './drive/MyDrive/MNIST/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
test_label_path =  './drive/MyDrive/MNIST/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte'

In [ ]:
train_images, train_labels = load_mnist(train_image_path, train_label_path)
test_images, test_labels = load_mnist(test_image_path, test_label_path)
val_images, val_labels = train_images[50000:], train_labels[50000:]
train_images, train_labels = train_images[:50000], train_labels[:50000]
print(f'Train images shape: {train_images.shape}')
print(f'Train labels shape: {train_labels.shape}')
print(f'Test images shape: {test_images.shape}')
print(f'Test labels shape: {test_labels.shape}')

#### Implementation of Resnet Backbone and AVG Aggregator for feature extraction

In [ ]:
import torchvision
import torch.nn as nn
import torch

class ResNet(nn.Module):
    def __init__(self,
                 model_name: str = 'resnet18',
                 pretrained: bool = True,
                 num_concepts: int = 19
                 ):
        super().__init__()
        self.model_name = model_name.lower()

        if pretrained:
            # The new naming of pretrained weights, you can change to V2 if desired.
            weights = 'IMAGENET1K_V1'
        else:
            weights = None

        self.model = torchvision.models.resnet18(weights=weights)

        # Modify the first convolutional layer to accept 1-channel input and adjust kernel size and stride
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)

        # Adjust max pooling layer to not downsample too much
        self.model.maxpool = nn.Identity()

        # Initialize the new conv1 layer properly
        nn.init.kaiming_normal_(self.model.conv1.weight, mode='fan_out', nonlinearity='relu')

        # num_ftrs = self.model.fc.in_features

        # Set the fully connected layer to identity
        self.model.fc = nn.Identity()

        # self.digit_classifer = nn.Linear(num_ftrs, num_concepts)

    def forward(self, x):
        x = self.model(x)
        # digit = self.digit_classifer(x)
        return x

In [ ]:
rn = ResNet()
rn(torch.randn(1, 1, 28, 56)).shape

#### Create MNISTDataset and MNISTAdditionDataset

In [ ]:
import numpy as np
import os
import struct
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = np.array(images, copy=True)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# MNIST transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# Create train, validation, and test datasets
train_dataset = MNISTDataset(train_images, train_labels, transform=transform)
val_dataset = MNISTDataset(val_images, val_labels, transform=transform)
test_dataset = MNISTDataset(test_images, test_labels,  transform=transform)

In [ ]:
train_dataset[0][0].shape , train_dataset[0][1]

In [ ]:
class MNISTAdditionDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        random_index = np.random.randint(len(self.dataset))

        image2, label2 = self.dataset[random_index]

        image = torch.cat((image, image2), dim=-1)#.permute(0, 3, 1, 2)

        c_label = torch.zeros((2,10))
        c_label[0][label] = 1
        c_label[1][label2] = 1

        y_label = label + label2

        return image, c_label, torch.tensor(y_label)


# Create the combined datasets
train_addition_dataset = MNISTAdditionDataset(train_dataset)
val_addition_dataset = MNISTAdditionDataset(val_dataset)
test_addition_dataset = MNISTAdditionDataset(test_dataset)

In [ ]:
train_addition_dataset[0][0].shape , train_addition_dataset[0][1]

In [ ]:

rn(train_addition_dataset[0][0].unsqueeze(1)).shape

In [ ]:
def prepare_data(dataset, model):
  x = []
  c = []
  y = []
  model.eval()
  with torch.no_grad():
    for image, c_label, y_label in dataset:
      image = image.unsqueeze(0) # Add batch dimension
      features = model(image)
      x.append(features.squeeze(0).numpy()) # Remove batch dimension and convert to numpy
      c.append(c_label.flatten().numpy())
      y.append(y_label.numpy())
  return np.array(x), np.array(c), np.array(y)

# Prepare data for training and testing
x_train, c_train, y_train = prepare_data(train_addition_dataset, rn)
#x_val, c_val, y_val = prepare_data(val_addition_dataset, rn)
x_test, c_test, y_test = prepare_data(test_addition_dataset, rn)


KeyboardInterrupt: 

In [ ]:
pwd

'/content'

In [ ]:
%cd /content/

/content


In [ ]:
import pickle

# Save the data
# with open('train_data.pkl', 'wb') as f:
#   pickle.dump((x_train, c_train, y_train), f)

# with open('test_data.pkl', 'wb') as f:
#   pickle.dump((x_test, c_test, y_test), f)

# Load the data
with open('train_data.pkl', 'rb') as f:
  x_train, c_train, y_train = pickle.load(f)

with open('test_data.pkl', 'rb') as f:
  x_test, c_test, y_test = pickle.load(f)

print(x_train.shape, c_train.shape, y_train.shape)


(50000, 512) (50000, 20) (50000,)


#### Let's start working

In [ ]:
!git clone -b code_exploration https://github.com/alialhousseini/pytorch_explain

Cloning into 'pytorch_explain'...
remote: Enumerating objects: 1318, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 1318 (delta 298), reused 406 (delta 291), pack-reused 890
Receiving objects: 100% (1318/1318), 8.35 MiB | 27.31 MiB/s, done.
Resolving deltas: 100% (764/764), done.


In [ ]:
%%capture
!pip install wandb

In [ ]:
%cd /content/pytorch_explain

/content/pytorch_explain


In [ ]:
import torch.nn.functional as F
from torch_explain.nn.concepts import IntpLinearLayer1, IntpLinearLayer2, IntpLinearLayer3, ConceptReasoningLayer, ConceptEmbedding
import torch
import torch_explain as te
from torch_explain import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb

In [ ]:
# Good practice: Keep all keys secret!
wandb.login(key='598e89dd4ece6531ed05c8fc9ea9bb6914798009')

In [ ]:
dataset_names = ['XOR','XNOR', 'IsBinEven','Trigonometry', 'Dot']

datasets = [
    datasets.xor(5000),
    datasets.xnor(5000),
    datasets.is_bin_even(5000),
    datasets.trigonometry(5000),
    datasets.dot(5000)
]

models = ['DCRBase', 'LLR1', 'LLR2', 'LLR3']

Train on MNIST

In [ ]:
# Function to select loss function based on the configuration
def get_loss_function(name):
    if name == 'bce':
        return torch.nn.BCELoss()
    elif name == 'mse':
        return torch.nn.MSELoss()
    elif name == 'huber':
        return torch.nn.HuberLoss()
    elif name == 'hinge':
        return torch.nn.HingeEmbeddingLoss()
    elif name == 'bceL':
        return torch.nn.BCEWithLogitsLoss()
    elif name == 'cross_entropy':
        return torch.nn.CrossEntropyLoss()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
y_train.shape

(50000,)

In [ ]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import wandb

print("===========================================================")
print("Training Started!")

# Define target accuracy for convergence time
target_accuracy = 0.5
isBias = True

# Iterate over models
for model_name in models:
    if model_name == 'DCRBase':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        dataset_name = "MNIST_Addition"
        print(f"The following dataset has been loaded successfully: {dataset_name}")

        x_train = torch.from_numpy(x_train).float()
        c_train = torch.from_numpy(c_train).float()
        y_train = torch.from_numpy(y_train).long()

        x_test = torch.from_numpy(x_test).float()
        c_test = torch.from_numpy(c_test).float()
        y_test = torch.from_numpy(y_test).long()

        y_train = F.one_hot(y_train.long().ravel()).float()
        y_test = F.one_hot(y_test.long().ravel()).float()

        embedding_size = 16
        concept_encoder = torch.nn.Sequential(
            torch.nn.Linear(x_train.shape[1], 32),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.LeakyReLU(),
            te.nn.ConceptEmbedding(32, c_train.shape[1], embedding_size),
        )

        task_predictor = ConceptReasoningLayer(embedding_size, y_train.shape[1])
        model = torch.nn.Sequential(concept_encoder, task_predictor)

        num_val_samples = int(len(x_train) * 0.2)
        num_train_samples = len(x_train) - num_val_samples
        train_dataset, val_dataset = random_split(
           list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                   name=f"{model_name}_{dataset_name}")

        config = {
            'lr': 0.0005,
            'task_loss_weight': 0.5,
            'loss_function': 'bce',
            'loss_function2': 'bceL',
            'loss_function3': 'cross_entropy',
        }
        wandb.config.update(config)

        loss_form = get_loss_function(wandb.config.loss_function3)
        optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

        print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

        # Initialize timers and convergence flag
        total_start_time = time.time()
        convergence_time = None

        for epoch in range(51):
            epoch_start_time = time.time()
            model.train()
            train_losses, train_correct = 0, 0
            all_y_true_train, all_y_pred_train = [], []
            all_c_true_train, all_c_pred_train = [], []

            for x_batch, c_batch, y_batch in train_loader:
                optimizer.zero_grad()
                c_emb, c_pred = concept_encoder(x_batch)
                y_pred = task_predictor(c_emb, c_pred)

                concept_loss = loss_form(c_pred, c_batch)
                task_loss = loss_form(y_pred, y_batch)
                loss = concept_loss + 0.5 * task_loss

                loss.backward()
                optimizer.step()

                train_losses += loss.item()
                train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                all_y_true_train.append(y_batch.cpu().numpy())
                all_y_pred_train.append(y_pred.detach().cpu().numpy())
                all_c_true_train.append(c_batch.cpu().numpy())
                all_c_pred_train.append(c_pred.detach().cpu().numpy())

                wandb.log({
                    'train_concept_loss': concept_loss.item(),
                    'train_task_loss': task_loss.item(),
                    'learning_rate': optimizer.param_groups[0]['lr']
                })

            all_y_true_train = np.concatenate(all_y_true_train, axis=0)
            all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
            all_c_true_train = np.concatenate(all_c_true_train, axis=0)
            all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)

            train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
            train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

            model.eval()
            val_losses, val_correct = 0, 0
            all_y_true_val, all_y_pred_val = [], []
            all_c_true_val, all_c_pred_val = [], []

            with torch.no_grad():
                for x_batch, c_batch, y_batch in val_loader:
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    val_concept_loss = loss_form(c_pred, c_batch)
                    val_task_loss = loss_form(y_pred, y_batch)
                    val_loss = val_concept_loss + 0.5 * val_task_loss

                    val_losses += val_loss.item()
                    val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_val.append(y_batch.cpu().numpy())
                    all_y_pred_val.append(y_pred.detach().cpu().numpy())
                    all_c_true_val.append(c_batch.cpu().numpy())
                    all_c_pred_val.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'val_concept_loss': val_concept_loss.item(),
                        'val_task_loss': val_task_loss.item(),
                        'val_learning_rate': optimizer.param_groups[0]['lr']
                    })

            all_y_true_val = np.concatenate(all_y_true_val, axis=0)
            all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
            all_c_true_val = np.concatenate(all_c_true_val, axis=0)
            all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)

            val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
            val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

            scheduler.step(val_losses / len(val_loader))

            epoch_end_time = time.time()
            epoch_time = epoch_end_time - epoch_start_time

            print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Train Concept Accuracy: {train_concept_accuracy}, Train Task Accuracy: {train_task_accuracy}, Val Concept Accuracy: {val_concept_accuracy}, Val Task Accuracy: {val_task_accuracy}, Epoch Time: {epoch_time}")

            wandb.log({
                'epoch': epoch + 1,
                'loss': train_losses / len(train_loader),
                'train_accuracy': train_correct / len(train_dataset),
                'val_loss': val_losses / len(val_loader),
                'val_accuracy': val_correct / len(val_dataset),
                'train_precision': train_precision,
                'train_recall': train_recall,
                'train_f1': train_f1,
                'val_precision': val_precision,
                'val_recall': val_recall,
                'val_f1': val_f1,
                'train_concept_accuracy': train_concept_accuracy,
                'train_task_accuracy': train_task_accuracy,
                'val_concept_accuracy': val_concept_accuracy,
                'val_task_accuracy': val_task_accuracy,
                'epoch_time': epoch_time
            })

            # Check for convergence
            if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                convergence_time = time.time() - total_start_time
                wandb.log({'convergence_time': convergence_time})
                print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

        total_training_time = time.time() - total_start_time
        wandb.log({'total_training_time': total_training_time})
        print(f"Total Training Time: {total_training_time}")

        print(f"\n Training on {dataset_name} using {model_name} has been completed!")
        torch.save(model, f'model_{model_name}_{dataset_name}.pth')
        torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
        wandb.finish()

        print(f"===========================================================")

    # Repeat similar updates for 'LLR1', 'LLR2', and 'LLR3' models


    elif model_name == 'LLR1':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        dataset_name= "MNIST_Addition"
        print(f"The following dataset has been loaded successfully: {dataset_name}")

        # y_train = F.one_hot(y_train.long().ravel()).float()
        # y_test = F.one_hot(y_test.long().ravel()).float()

        embedding_size = 16
        concept_encoder = torch.nn.Sequential(
            torch.nn.Linear(x_train.shape[1], 32),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.LeakyReLU(),
            te.nn.ConceptEmbedding(32, c_train.shape[1], embedding_size),
        )

        task_predictor = IntpLinearLayer1(embedding_size, y_train.shape[1], bias=isBias)
        model = torch.nn.Sequential(concept_encoder, task_predictor)

        num_val_samples = int(len(x_train) * 0.2)
        num_train_samples = len(x_train) - num_val_samples
        train_dataset, val_dataset = random_split(
            list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                   name=f"{model_name}_{dataset_name}")

        config = {
            'lr': 0.0005,
            'task_loss_weight': 0.5,
            'loss_function': 'bce',
            'loss_function2': 'bceL',
            'loss_function3': 'cross_entropy',
        }
        wandb.config.update(config)

        c_loss = get_loss_function(wandb.config.loss_function)
        y_loss = get_loss_function(wandb.config.loss_function3)

        optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

        print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

        # Initialize timers and convergence flag
        total_start_time = time.time()
        convergence_time = None

        for epoch in range(41):
            epoch_start_time = time.time()
            model.train()
            train_losses, train_correct = 0, 0
            all_y_true_train, all_y_pred_train = [], []
            all_c_true_train, all_c_pred_train = [], []

            for x_batch, c_batch, y_batch in train_loader:
                optimizer.zero_grad()
                c_emb, c_pred = concept_encoder(x_batch)
                y_pred = task_predictor(c_emb, c_pred)

                concept_loss = c_loss(c_pred, c_batch)
                task_loss = y_loss(y_pred, y_batch)
                loss = concept_loss + 0.5 * task_loss

                loss.backward()
                optimizer.step()

                train_losses += loss.item()
                train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                all_y_true_train.append(y_batch.cpu().numpy())
                all_y_pred_train.append(y_pred.detach().cpu().numpy())
                all_c_true_train.append(c_batch.cpu().numpy())
                all_c_pred_train.append(c_pred.detach().cpu().numpy())

                wandb.log({
                    'train_concept_loss': concept_loss.item(),
                    'train_task_loss': task_loss.item(),
                    'learning_rate': optimizer.param_groups[0]['lr']
                })

            all_y_true_train = np.concatenate(all_y_true_train, axis=0)
            all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
            all_c_true_train = np.concatenate(all_c_true_train, axis=0)
            all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)

            train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
            train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

            model.eval()
            val_losses, val_correct = 0, 0
            all_y_true_val, all_y_pred_val = [], []
            all_c_true_val, all_c_pred_val = [], []

            with torch.no_grad():
                for x_batch, c_batch, y_batch in val_loader:
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    val_concept_loss = c_loss(c_pred, c_batch)
                    val_task_loss = y_loss(y_pred, y_batch)
                    val_loss = val_concept_loss + 0.5 * val_task_loss

                    val_losses += val_loss.item()
                    val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_val.append(y_batch.cpu().numpy())
                    all_y_pred_val.append(y_pred.detach().cpu().numpy())
                    all_c_true_val.append(c_batch.cpu().numpy())
                    all_c_pred_val.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'val_concept_loss': val_concept_loss.item(),
                        'val_task_loss': val_task_loss.item(),
                        'val_learning_rate': optimizer.param_groups[0]['lr']
                    })

            all_y_true_val = np.concatenate(all_y_true_val, axis=0)
            all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
            all_c_true_val = np.concatenate(all_c_true_val, axis=0)
            all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)

            val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
            val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

            scheduler.step(val_losses / len(val_loader))

            epoch_end_time = time.time()
            epoch_time = epoch_end_time - epoch_start_time

            print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

            wandb.log({
                'epoch': epoch + 1,
                'loss': train_losses / len(train_loader),
                'train_accuracy': train_correct / len(train_dataset),
                'val_loss': val_losses / len(val_loader),
                'val_accuracy': val_correct / len(val_dataset),
                'train_precision': train_precision,
                'train_recall': train_recall,
                'train_f1': train_f1,
                'val_precision': val_precision,
                'val_recall': val_recall,
                'val_f1': val_f1,
                'train_concept_accuracy': train_concept_accuracy,
                'train_task_accuracy': train_task_accuracy,
                'val_concept_accuracy': val_concept_accuracy,
                'val_task_accuracy': val_task_accuracy,
                'epoch_time': epoch_time
            })

            # Check for convergence
            if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                convergence_time = time.time() - total_start_time
                wandb.log({'convergence_time': convergence_time})
                print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

        total_training_time = time.time() - total_start_time
        wandb.log({'total_training_time': total_training_time})
        print(f"Total Training Time: {total_training_time}")

        print(f"\n Training on {dataset_name} using {model_name} has been completed!")
        torch.save(model, f'model_{model_name}_{dataset_name}.pth')
        torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
        wandb.finish()

        print(f"===========================================================")

    elif model_name == 'LLR2':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")
        print(f"The following dataset has been loaded successfully: {dataset_name}")

        # y_train = F.one_hot(y_train.long().ravel()).float()
        # y_test = F.one_hot(y_test.long().ravel()).float()

        embedding_size = 16
        concept_encoder = torch.nn.Sequential(
            torch.nn.Linear(x_train.shape[1], 32),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.LeakyReLU(),
            te.nn.ConceptEmbedding(32, c_train.shape[1], embedding_size),
        )

        task_predictor = IntpLinearLayer2(embedding_size, y_train.shape[1], bias=isBias)
        model = torch.nn.Sequential(concept_encoder, task_predictor)

        wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                   name=f"{model_name}_{dataset_name}")

        config = {
            'lr': 0.0005,
            'task_loss_weight': 0.5,
            'loss_function': 'bce',
            'loss_function2': 'bceL',
            'loss_function3': 'cross_entropy',
        }
        wandb.config.update(config)

        c_loss = get_loss_function(wandb.config.loss_function)
        y_loss = get_loss_function(wandb.config.loss_function3)

        optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

        print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

        # Initialize timers and convergence flag
        total_start_time = time.time()
        convergence_time = None

        for epoch in range(41):
            epoch_start_time = time.time()
            model.train()
            train_losses, train_correct = 0, 0
            all_y_true_train, all_y_pred_train = [], []
            all_c_true_train, all_c_pred_train = [], []

            for x_batch, c_batch, y_batch in train_loader:
                optimizer.zero_grad()
                c_emb, c_pred = concept_encoder(x_batch)
                y_pred = task_predictor(c_emb, c_pred)

                concept_loss = c_loss(c_pred, c_batch)
                task_loss = y_loss(y_pred, y_batch)
                loss = concept_loss + 0.5 * task_loss

                loss.backward()
                optimizer.step()

                train_losses += loss.item()
                train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                all_y_true_train.append(y_batch.cpu().numpy())
                all_y_pred_train.append(y_pred.detach().cpu().numpy())
                all_c_true_train.append(c_batch.cpu().numpy())
                all_c_pred_train.append(c_pred.detach().cpu().numpy())

                wandb.log({
                    'train_concept_loss': concept_loss.item(),
                    'train_task_loss': task_loss.item(),
                    'learning_rate': optimizer.param_groups[0]['lr']
                })

            all_y_true_train = np.concatenate(all_y_true_train, axis=0)
            all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
            all_c_true_train = np.concatenate(all_c_true_train, axis=0)
            all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)

            train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
            train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

            model.eval()
            val_losses, val_correct = 0, 0
            all_y_true_val, all_y_pred_val = [], []
            all_c_true_val, all_c_pred_val = [], []

            with torch.no_grad():
                for x_batch, c_batch, y_batch in val_loader:
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    val_concept_loss = c_loss(c_pred, c_batch)
                    val_task_loss = y_loss(y_pred, y_batch)
                    val_loss = val_concept_loss + 0.5 * val_task_loss

                    val_losses += val_loss.item()
                    val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_val.append(y_batch.cpu().numpy())
                    all_y_pred_val.append(y_pred.detach().cpu().numpy())
                    all_c_true_val.append(c_batch.cpu().numpy())
                    all_c_pred_val.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'val_concept_loss': val_concept_loss.item(),
                        'val_task_loss': val_task_loss.item(),
                        'val_learning_rate': optimizer.param_groups[0]['lr']
                    })


            all_y_true_val = np.concatenate(all_y_true_val, axis=0)
            all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
            all_c_true_val = np.concatenate(all_c_true_val, axis=0)
            all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)

            val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
            val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

            scheduler.step(val_losses / len(val_loader))

            epoch_end_time = time.time()
            epoch_time = epoch_end_time - epoch_start_time

            print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

            wandb.log({
                'epoch': epoch + 1,
                'loss': train_losses / len(train_loader),
                'train_accuracy': train_correct / len(train_dataset),
                'val_loss': val_losses / len(val_loader),
                'val_accuracy': val_correct / len(val_dataset),
                'train_precision': train_precision,
                'train_recall': train_recall,
                'train_f1': train_f1,
                'val_precision': val_precision,
                'val_recall': val_recall,
                'val_f1': val_f1,
                'train_concept_accuracy': train_concept_accuracy,
                'train_task_accuracy': train_task_accuracy,
                'val_concept_accuracy': val_concept_accuracy,
                'val_task_accuracy': val_task_accuracy,
                'epoch_time': epoch_time
            })

            # Check for convergence
            if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                convergence_time = time.time() - total_start_time
                wandb.log({'convergence_time': convergence_time})
                print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

        total_training_time = time.time() - total_start_time
        wandb.log({'total_training_time': total_training_time})
        print(f"Total Training Time: {total_training_time}")

        print(f"\n Training on {dataset_name} using {model_name} has been completed!")
        torch.save(model, f'model_{model_name}_{dataset_name}.pth')
        torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
        wandb.finish()

        print(f"===========================================================")

    if model_name == 'LLR3':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        embedding_size = 16
        concept_encoder = torch.nn.Sequential(
            torch.nn.Linear(x_train.shape[1], 32),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.LeakyReLU(),
            te.nn.ConceptEmbedding(32, c_train.shape[1], embedding_size),
        )

        task_predictor = IntpLinearLayer3(embedding_size, y_train.shape[1], bias=isBias)
        model = torch.nn.Sequential(concept_encoder, task_predictor)

        # num_val_samples = int(len(x_train) * 0.2)
        # num_train_samples = len(x_train) - num_val_samples
        # train_dataset, val_dataset = random_split(
        #     list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

        # train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        # val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                   name=f"{model_name}_{dataset_name}")

        config = {
            'lr': 0.0005,
            'task_loss_weight': 0.5,
            'loss_function': 'bce',
            'loss_function2': 'bceL',
            'loss_function3': 'cross_entropy',
        }
        wandb.config.update(config)

        c_loss = get_loss_function(wandb.config.loss_function)
        y_loss = get_loss_function(wandb.config.loss_function3)

        optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

        print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

        # Initialize timers and convergence flag
        total_start_time = time.time()
        convergence_time = None

        for epoch in range(101):
            epoch_start_time = time.time()
            model.train()
            train_losses, train_correct = 0, 0
            all_y_true_train, all_y_pred_train = [], []
            all_c_true_train, all_c_pred_train = [], []

            for x_batch, c_batch, y_batch in train_loader:
                optimizer.zero_grad()
                c_emb, c_pred = concept_encoder(x_batch)
                y_pred = task_predictor(c_emb, c_pred)

                concept_loss = c_loss(c_pred, c_batch)
                task_loss = y_loss(y_pred, y_batch)
                loss = concept_loss + 0.5 * task_loss

                loss.backward()
                optimizer.step()

                train_losses += loss.item()
                train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                all_y_true_train.append(y_batch.cpu().numpy())
                all_y_pred_train.append(y_pred.detach().cpu().numpy())
                all_c_true_train.append(c_batch.cpu().numpy())
                all_c_pred_train.append(c_pred.detach().cpu().numpy())

                wandb.log({
                    'train_concept_loss': concept_loss.item(),
                    'train_task_loss': task_loss.item(),
                    'learning_rate': optimizer.param_groups[0]['lr']
                })

            all_y_true_train = np.concatenate(all_y_true_train, axis=0)
            all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
            all_c_true_train = np.concatenate(all_c_true_train, axis=0)
            all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)

            train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
            train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
            train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

            model.eval()
            val_losses, val_correct = 0, 0
            all_y_true_val, all_y_pred_val = [], []
            all_c_true_val, all_c_pred_val = [], []

            with torch.no_grad():
                for x_batch, c_batch, y_batch in val_loader:
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    val_concept_loss = c_loss(c_pred, c_batch)
                    val_task_loss = y_loss(y_pred, y_batch)
                    val_loss = val_concept_loss + 0.5 * val_task_loss

                    val_losses += val_loss.item()
                    val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_val.append(y_batch.cpu().numpy())
                    all_y_pred_val.append(y_pred.detach().cpu().numpy())
                    all_c_true_val.append(c_batch.cpu().numpy())
                    all_c_pred_val.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'val_concept_loss': val_concept_loss.item(),
                        'val_task_loss': val_task_loss.item(),
                        'val_learning_rate': optimizer.param_groups[0]['lr']
                    })

            all_y_true_val = np.concatenate(all_y_true_val, axis=0)
            all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
            all_c_true_val = np.concatenate(all_c_true_val, axis=0)
            all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)
            val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
            val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
            val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

            scheduler.step(val_losses / len(val_loader))

            epoch_end_time = time.time()
            epoch_time = epoch_end_time - epoch_start_time

            print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

            wandb.log({
                'epoch': epoch + 1,
                'loss': train_losses / len(train_loader),
                'train_accuracy': train_correct / len(train_dataset),
                'val_loss': val_losses / len(val_loader),
                'val_accuracy': val_correct / len(val_dataset),
                'train_precision': train_precision,
                'train_recall': train_recall,
                'train_f1': train_f1,
                'val_precision': val_precision,
                'val_recall': val_recall,
                'val_f1': val_f1,
                'train_concept_accuracy': train_concept_accuracy,
                'train_task_accuracy': train_task_accuracy,
                'val_concept_accuracy': val_concept_accuracy,
                'val_task_accuracy': val_task_accuracy,
                'epoch_time': epoch_time
            })


            # Check for convergence
            if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                convergence_time = time.time() - total_start_time
                wandb.log({'convergence_time': convergence_time})
                print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

        total_training_time = time.time() - total_start_time
        wandb.log({'total_training_time': total_training_time})
        print(f"Total Training Time: {total_training_time}")

        print(f"\n Training on {dataset_name} using {model_name} has been completed!")
        torch.save(model, f'model_{model_name}_{dataset_name}.pth')
        torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
        wandb.finish()

        print(f"===========================================================")


print(f"*********** ALL TRAINING ARE DONE - Check WandB ***********")


Training Started!
Training on DCRBase ... 
--------------------------------
The following dataset has been loaded successfully: MNIST_Addition
Training on LLR3 ... 
--------------------------------


learning_rate,▁▁▁▁▁▁▁▁▁▁▁
train_concept_loss,██▇▆▆▅▄▄▃▂▁
train_task_loss,▄██▅▁▃▅▂▂▃▆
learning_rate,0.0005
train_concept_loss,0.6629
train_task_loss,3.19551


-------------------------- Training MNIST_Addition using LLR3 ----------------------
Epoch 1, Loss: 1.7462483808517455, Train Accuracy: 0.0989, Val Loss: 1.6840228852193067, Val Accuracy: 0.0892, Train Precision: 0.031598877364128954, Train Recall: 0.0989, Train F1: 0.036714184199069445, Val Precision: 0.0213018437637755, Val Recall: 0.0892, Val F1: 0.02751930180610301, Epoch Time: 16.853581428527832
Epoch 2, Loss: 1.5685187818527222, Train Accuracy: 0.157675, Val Loss: 1.4384291604825645, Val Accuracy: 0.2166, Train Precision: 0.18838971451170192, Train Recall: 0.157675, Train F1: 0.12245993239323015, Val Precision: 0.21813463613356512, Val Recall: 0.2166, Val F1: 0.18281724653663914, Epoch Time: 17.98530626296997
Epoch 3, Loss: 1.379292474937439, Train Accuracy: 0.2432, Val Loss: 1.3366748246417683, Val Accuracy: 0.2626, Train Precision: 0.24010235236504973, Train Recall: 0.2432, Train F1: 0.22153339361655336, Val Precision: 0.26843970959001107, Val Recall: 0.2626, Val F1: 0.23010103

convergence_time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_time,▁▁▁▁▂▂▁▁█▃▁▂▁▁▁▁▂▁▂▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▂▁▂▃▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
train_concept_accuracy,▁▂▂▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇████████████████
train_concept_loss,█▇▇▇▆▆▆▆▆▅▅▆▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▂▁▁▁
train_f1,▁▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████
train_precision,▁▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████


*********** ALL TRAINING ARE DONE - Check WandB ***********


In [ ]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score

print("===========================================================")
print("Training Started!")

# Define target accuracy for convergence time
target_accuracy = 0.9  # for example

# Iterate over models
for model_name in models:
    if model_name == 'DCRBase':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        for dataset_name, dataset in zip(dataset_names, datasets):
            x, c, y = dataset
            x_train, x_test, c_train, c_test, y_train, y_test = train_test_split(
                x, c, y, test_size=0.3, random_state=42)
            print(f"The following dataset has been loaded successfully: {dataset_name}")

            y_train = F.one_hot(y_train.long().ravel()).float()
            y_test = F.one_hot(y_test.long().ravel()).float()

            embedding_size = 16
            concept_encoder = torch.nn.Sequential(
                torch.nn.Linear(x.shape[1], 16),
                torch.nn.LeakyReLU(),
                te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
            )

            task_predictor = ConceptReasoningLayer(embedding_size, y_train.shape[1])
            model = torch.nn.Sequential(concept_encoder, task_predictor)

            num_val_samples = int(len(x_train) * 0.2)
            num_train_samples = len(x_train) - num_val_samples
            train_dataset, val_dataset = random_split(
                list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

            wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                       name=f"{model_name}_{dataset_name}")

            config = {
                'lr': 0.0005,
                'task_loss_weight': 0.5,
                'loss_function': 'bce',
                'loss_function2': 'bceL',
                'loss_function3': 'cross_entropy'
            }
            wandb.config.update(config)

            loss_form = get_loss_function(wandb.config.loss_function)
            optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

            print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

            # Initialize timers and convergence flag
            total_start_time = time.time()
            convergence_time = None

            for epoch in range(41):
                epoch_start_time = time.time()
                model.train()
                train_losses, train_correct = 0, 0
                all_y_true_train, all_y_pred_train = [], []
                all_c_true_train, all_c_pred_train = [], []

                for x_batch, c_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    concept_loss = loss_form(c_pred, c_batch)
                    task_loss = loss_form(y_pred, y_batch)
                    loss = concept_loss + 0.5 * task_loss

                    loss.backward()
                    optimizer.step()

                    train_losses += loss.item()
                    train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_train.append(y_batch.cpu().numpy())
                    all_y_pred_train.append(y_pred.detach().cpu().numpy())
                    all_c_true_train.append(c_batch.cpu().numpy())
                    all_c_pred_train.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'train_concept_loss': concept_loss.item(),
                        'train_task_loss': task_loss.item(),
                        'learning_rate': optimizer.param_groups[0]['lr']
                    })

                all_y_true_train = np.concatenate(all_y_true_train, axis=0)
                all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
                all_c_true_train = np.concatenate(all_c_true_train, axis=0)
                all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)
                train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
                train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

                model.eval()
                val_losses, val_correct = 0, 0
                all_y_true_val, all_y_pred_val = [], []
                all_c_true_val, all_c_pred_val = [], []

                with torch.no_grad():
                    for x_batch, c_batch, y_batch in val_loader:
                        c_emb, c_pred = concept_encoder(x_batch)
                        y_pred = task_predictor(c_emb, c_pred)

                        val_concept_loss = loss_form(c_pred, c_batch)
                        val_task_loss = loss_form(y_pred, y_batch)
                        val_loss = val_concept_loss + 0.5 * val_task_loss

                        val_losses += val_loss.item()
                        val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                        all_y_true_val.append(y_batch.cpu().numpy())
                        all_y_pred_val.append(y_pred.detach().cpu().numpy())
                        all_c_true_val.append(c_batch.cpu().numpy())
                        all_c_pred_val.append(c_pred.detach().cpu().numpy())

                        wandb.log({
                            'val_concept_loss': val_concept_loss.item(),
                            'val_task_loss': val_task_loss.item(),
                            'val_learning_rate': optimizer.param_groups[0]['lr']
                        })

                all_y_true_val = np.concatenate(all_y_true_val, axis=0)
                all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
                all_c_true_val = np.concatenate(all_c_true_val, axis=0)
                all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)
                val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
                val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

                scheduler.step(val_losses / len(val_loader))

                epoch_end_time = time.time()
                epoch_time = epoch_end_time - epoch_start_time

                print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

                wandb.log({
                    'epoch': epoch + 1,
                    'loss': train_losses / len(train_loader),
                    'train_accuracy': train_correct / len(train_dataset),
                    'val_loss': val_losses / len(val_loader),
                    'val_accuracy': val_correct / len(val_dataset),
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'val_precision': val_precision,
                    'val_recall': val_recall,
                    'val_f1': val_f1,
                    'train_concept_accuracy': train_concept_accuracy,
                    'train_task_accuracy': train_task_accuracy,
                    'val_concept_accuracy': val_concept_accuracy,
                    'val_task_accuracy': val_task_accuracy,
                    'epoch_time': epoch_time
                })

                # Check for convergence
                if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                    convergence_time = time.time() - total_start_time
                    wandb.log({'convergence_time': convergence_time})
                    print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

            total_training_time = time.time() - total_start_time
            wandb.log({'total_training_time': total_training_time})
            print(f"Total Training Time: {total_training_time}")

            print(f"\n Training on {dataset_name} using {model_name} has been completed!")
            torch.save(model, f'model_{model_name}_{dataset_name}.pth')
            torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
            wandb.finish()

        print(f"===========================================================")

    if model_name == 'LLR1':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        for dataset_name, dataset in zip(dataset_names, datasets):
            x, c, y = dataset
            x_train, x_test, c_train, c_test, y_train, y_test = train_test_split(
                x, c, y, test_size=0.3, random_state=42)
            print(f"The following dataset has been loaded successfully: {dataset_name}")

            y_train = F.one_hot(y_train.long().ravel()).float()
            y_test = F.one_hot(y_test.long().ravel()).float()

            embedding_size = 16
            concept_encoder = torch.nn.Sequential(
                torch.nn.Linear(x.shape[1], 16),
                torch.nn.LeakyReLU(),
                te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
            )

            task_predictor = IntpLinearLayer1(embedding_size, y_train.shape[1], bias=isBias)
            model = torch.nn.Sequential(concept_encoder, task_predictor)

            num_val_samples = int(len(x_train) * 0.2)
            num_train_samples = len(x_train) - num_val_samples
            train_dataset, val_dataset = random_split(
                list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

            wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                       name=f"{model_name}_{dataset_name}")

            config = {
                'lr': 0.0005,
                'task_loss_weight': 0.5,
                'loss_function': 'bce',
                'loss_function2': 'bceL',
                'loss_function3': 'cross_entropy',
            }
            wandb.config.update(config)

            c_loss = get_loss_function(wandb.config.loss_function2)
            y_loss = get_loss_function(wandb.config.loss_function2)

            optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

            print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

            # Initialize timers and convergence flag
            total_start_time = time.time()
            convergence_time = None

            for epoch in range(41):
                epoch_start_time = time.time()
                model.train()
                train_losses, train_correct = 0, 0
                all_y_true_train, all_y_pred_train = [], []
                all_c_true_train, all_c_pred_train = [], []

                for x_batch, c_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    concept_loss = c_loss(c_pred, c_batch)
                    task_loss = y_loss(y_pred, y_batch)
                    loss = concept_loss + 0.5 * task_loss

                    loss.backward()
                    optimizer.step()

                    train_losses += loss.item()
                    train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_train.append(y_batch.cpu().numpy())
                    all_y_pred_train.append(y_pred.detach().cpu().numpy())
                    all_c_true_train.append(c_batch.cpu().numpy())
                    all_c_pred_train.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'train_concept_loss': concept_loss.item(),
                        'train_task_loss': task_loss.item(),
                        'learning_rate': optimizer.param_groups[0]['lr']
                    })

                all_y_true_train = np.concatenate(all_y_true_train, axis=0)
                all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
                all_c_true_train = np.concatenate(all_c_true_train, axis=0)
                all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)
                train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
                train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

                model.eval()
                val_losses, val_correct = 0, 0
                all_y_true_val, all_y_pred_val = [], []
                all_c_true_val, all_c_pred_val = [], []

                with torch.no_grad():
                    for x_batch, c_batch, y_batch in val_loader:
                        c_emb, c_pred = concept_encoder(x_batch)
                        y_pred = task_predictor(c_emb, c_pred)

                        val_concept_loss = c_loss(c_pred, c_batch)
                        val_task_loss = y_loss(y_pred, y_batch)
                        val_loss = val_concept_loss + 0.5 * val_task_loss

                        val_losses += val_loss.item()
                        val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                        all_y_true_val.append(y_batch.cpu().numpy())
                        all_y_pred_val.append(y_pred.detach().cpu().numpy())
                        all_c_true_val.append(c_batch.cpu().numpy())
                        all_c_pred_val.append(c_pred.detach().cpu().numpy())

                        wandb.log({
                            'val_concept_loss': val_concept_loss.item(),
                            'val_task_loss': val_task_loss.item(),
                            'val_learning_rate': optimizer.param_groups[0]['lr']
                        })

                all_y_true_val = np.concatenate(all_y_true_val, axis=0)
                all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
                all_c_true_val = np.concatenate(all_c_true_val, axis=0)
                all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)
                val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
                val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

                scheduler.step(val_losses / len(val_loader))

                epoch_end_time = time.time()
                epoch_time = epoch_end_time - epoch_start_time

                print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

                wandb.log({
                    'epoch': epoch + 1,
                    'loss': train_losses / len(train_loader),
                    'train_accuracy': train_correct / len(train_dataset),
                    'val_loss': val_losses / len(val_loader),
                    'val_accuracy': val_correct / len(val_dataset),
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'val_precision': val_precision,
                    'val_recall': val_recall,
                    'val_f1': val_f1,
                    'train_concept_accuracy': train_concept_accuracy,
                    'train_task_accuracy': train_task_accuracy,
                    'val_concept_accuracy': val_concept_accuracy,
                    'val_task_accuracy': val_task_accuracy,
                    'epoch_time': epoch_time
                })

                # Check for convergence
                if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                    convergence_time = time.time() - total_start_time
                    wandb.log({'convergence_time': convergence_time})
                    print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

            total_training_time = time.time() - total_start_time
            wandb.log({'total_training_time': total_training_time})
            print(f"Total Training Time: {total_training_time}")

            print(f"\n Training on {dataset_name} using {model_name} has been completed!")
            torch.save(model, f'model_{model_name}_{dataset_name}.pth')
            torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
            wandb.finish()

        print(f"===========================================================")

    if model_name == 'LLR2':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        for dataset_name, dataset in zip(dataset_names, datasets):
            x, c, y = dataset
            x_train, x_test, c_train, c_test, y_train, y_test = train_test_split(
                x, c, y, test_size=0.3, random_state=42)
            print(f"The following dataset has been loaded successfully: {dataset_name}")

            y_train = F.one_hot(y_train.long().ravel()).float()
            y_test = F.one_hot(y_test.long().ravel()).float()

            embedding_size = 16
            concept_encoder = torch.nn.Sequential(
                torch.nn.Linear(x.shape[1], 16),
                torch.nn.LeakyReLU(),
                te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
            )

            task_predictor = IntpLinearLayer2(embedding_size, y_train.shape[1], bias=isBias)
            model = torch.nn.Sequential(concept_encoder, task_predictor)

            num_val_samples = int(len(x_train) * 0.2)
            num_train_samples = len(x_train) - num_val_samples
            train_dataset, val_dataset = random_split(
                list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

            wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                       name=f"{model_name}_{dataset_name}")

            config = {
                'lr': 0.0005,
                'task_loss_weight': 0.5,
                'loss_function': 'bce',
                'loss_function2': 'bceL',
            }
            wandb.config.update(config)

            c_loss = get_loss_function(wandb.config.loss_function2)
            y_loss = get_loss_function(wandb.config.loss_function2)

            optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

            print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

            # Initialize timers and convergence flag
            total_start_time = time.time()
            convergence_time = None

            for epoch in range(41):
                epoch_start_time = time.time()
                model.train()
                train_losses, train_correct = 0, 0
                all_y_true_train, all_y_pred_train = [], []
                all_c_true_train, all_c_pred_train = [], []

                for x_batch, c_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    concept_loss = c_loss(c_pred, c_batch)
                    task_loss = y_loss(y_pred, y_batch)
                    loss = concept_loss + 0.5 * task_loss

                    loss.backward()
                    optimizer.step()

                    train_losses += loss.item()
                    train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_train.append(y_batch.cpu().numpy())
                    all_y_pred_train.append(y_pred.detach().cpu().numpy())
                    all_c_true_train.append(c_batch.cpu().numpy())
                    all_c_pred_train.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'train_concept_loss': concept_loss.item(),
                        'train_task_loss': task_loss.item(),
                        'learning_rate': optimizer.param_groups[0]['lr']
                    })

                all_y_true_train = np.concatenate(all_y_true_train, axis=0)
                all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
                all_c_true_train = np.concatenate(all_c_true_train, axis=0)
                all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)
                train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
                train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

                model.eval()
                val_losses, val_correct = 0, 0
                all_y_true_val, all_y_pred_val = [], []
                all_c_true_val, all_c_pred_val = [], []

                with torch.no_grad():
                    for x_batch, c_batch, y_batch in val_loader:
                        c_emb, c_pred = concept_encoder(x_batch)
                        y_pred = task_predictor(c_emb, c_pred)

                        val_concept_loss = c_loss(c_pred, c_batch)
                        val_task_loss = y_loss(y_pred, y_batch)
                        val_loss = val_concept_loss + 0.5 * val_task_loss

                        val_losses += val_loss.item()
                        val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                        all_y_true_val.append(y_batch.cpu().numpy())
                        all_y_pred_val.append(y_pred.detach().cpu().numpy())
                        all_c_true_val.append(c_batch.cpu().numpy())
                        all_c_pred_val.append(c_pred.detach().cpu().numpy())

                        wandb.log({
                            'val_concept_loss': val_concept_loss.item(),
                            'val_task_loss': val_task_loss.item(),
                            'val_learning_rate': optimizer.param_groups[0]['lr']
                        })

                all_y_true_val = np.concatenate(all_y_true_val, axis=0)
                all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
                all_c_true_val = np.concatenate(all_c_true_val, axis=0)
                all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)
                val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
                val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

                scheduler.step(val_losses / len(val_loader))

                epoch_end_time = time.time()
                epoch_time = epoch_end_time - epoch_start_time

                print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

                wandb.log({
                    'epoch': epoch + 1,
                    'loss': train_losses / len(train_loader),
                    'train_accuracy': train_correct / len(train_dataset),
                    'val_loss': val_losses / len(val_loader),
                    'val_accuracy': val_correct / len(val_dataset),
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'val_precision': val_precision,
                    'val_recall': val_recall,
                    'val_f1': val_f1,
                    'train_concept_accuracy': train_concept_accuracy,
                    'train_task_accuracy': train_task_accuracy,
                    'val_concept_accuracy': val_concept_accuracy,
                    'val_task_accuracy': val_task_accuracy,
                    'epoch_time': epoch_time
                })

                # Check for convergence
                if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                    convergence_time = time.time() - total_start_time
                    wandb.log({'convergence_time': convergence_time})
                    print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

            total_training_time = time.time() - total_start_time
            wandb.log({'total_training_time': total_training_time})
            print(f"Total Training Time: {total_training_time}")

            print(f"\n Training on {dataset_name} using {model_name} has been completed!")
            torch.save(model, f'model_{model_name}_{dataset_name}.pth')
            torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
            wandb.finish()

        print(f"===========================================================")


    if model_name == 'LLR3':
        print(f"Training on {model_name} ... ")
        print(f"--------------------------------")

        for dataset_name, dataset in zip(dataset_names, datasets):
            x, c, y = dataset
            x_train, x_test, c_train, c_test, y_train, y_test = train_test_split(
                x, c, y, test_size=0.3, random_state=42)
            print(f"The following dataset has been loaded successfully: {dataset_name}")

            y_train = F.one_hot(y_train.long().ravel()).float()
            y_test = F.one_hot(y_test.long().ravel()).float()

            embedding_size = 16
            concept_encoder = torch.nn.Sequential(
                torch.nn.Linear(x.shape[1], 16),
                torch.nn.LeakyReLU(),
                te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
            )

            task_predictor = IntpLinearLayer3(embedding_size, y_train.shape[1], bias=isBias)
            model = torch.nn.Sequential(concept_encoder, task_predictor)

            num_val_samples = int(len(x_train) * 0.2)
            num_train_samples = len(x_train) - num_val_samples
            train_dataset, val_dataset = random_split(
                list(zip(x_train, c_train, y_train)), [num_train_samples, num_val_samples])

            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

            wandb.init(project="pytorch_explain", entity="alih999954-politecnico-di-torino",
                       name=f"{model_name}_{dataset_name}")

            config = {
                'lr': 0.0005,
                'task_loss_weight': 0.5,
                'loss_function': 'bce',
                'loss_function2': 'bceL',
            }
            wandb.config.update(config)

            c_loss = get_loss_function(wandb.config.loss_function2)
            y_loss = get_loss_function(wandb.config.loss_function2)

            optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.lr)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.0001, patience=7)

            print(f'-------------------------- Training {dataset_name} using {model_name} ----------------------')

            # Initialize timers and convergence flag
            total_start_time = time.time()
            convergence_time = None

            for epoch in range(41):
                epoch_start_time = time.time()
                model.train()
                train_losses, train_correct = 0, 0
                all_y_true_train, all_y_pred_train = [], []
                all_c_true_train, all_c_pred_train = [], []

                for x_batch, c_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    c_emb, c_pred = concept_encoder(x_batch)
                    y_pred = task_predictor(c_emb, c_pred)

                    concept_loss = c_loss(c_pred, c_batch)
                    task_loss = y_loss(y_pred, y_batch)
                    loss = concept_loss + 0.5 * task_loss

                    loss.backward()
                    optimizer.step()

                    train_losses += loss.item()
                    train_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                    all_y_true_train.append(y_batch.cpu().numpy())
                    all_y_pred_train.append(y_pred.detach().cpu().numpy())
                    all_c_true_train.append(c_batch.cpu().numpy())
                    all_c_pred_train.append(c_pred.detach().cpu().numpy())

                    wandb.log({
                        'train_concept_loss': concept_loss.item(),
                        'train_task_loss': task_loss.item(),
                        'learning_rate': optimizer.param_groups[0]['lr']
                    })

                all_y_true_train = np.concatenate(all_y_true_train, axis=0)
                all_y_pred_train = np.concatenate(all_y_pred_train, axis=0)
                all_c_true_train = np.concatenate(all_c_true_train, axis=0)
                all_c_pred_train = np.concatenate(all_c_pred_train, axis=0)
                train_precision = precision_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_recall = recall_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_f1 = f1_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1), average='weighted')
                train_concept_accuracy = accuracy_score(all_c_true_train.argmax(1), all_c_pred_train.argmax(1))
                train_task_accuracy = accuracy_score(all_y_true_train.argmax(1), all_y_pred_train.argmax(1))

                model.eval()
                val_losses, val_correct = 0, 0
                all_y_true_val, all_y_pred_val = [], []
                all_c_true_val, all_c_pred_val = [], []

                with torch.no_grad():
                    for x_batch, c_batch, y_batch in val_loader:
                        c_emb, c_pred = concept_encoder(x_batch)
                        y_pred = task_predictor(c_emb, c_pred)

                        val_concept_loss = c_loss(c_pred, c_batch)
                        val_task_loss = y_loss(y_pred, y_batch)
                        val_loss = val_concept_loss + 0.5 * val_task_loss

                        val_losses += val_loss.item()
                        val_correct += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
                        all_y_true_val.append(y_batch.cpu().numpy())
                        all_y_pred_val.append(y_pred.detach().cpu().numpy())
                        all_c_true_val.append(c_batch.cpu().numpy())
                        all_c_pred_val.append(c_pred.detach().cpu().numpy())

                        wandb.log({
                            'val_concept_loss': val_concept_loss.item(),
                            'val_task_loss': val_task_loss.item(),
                            'val_learning_rate': optimizer.param_groups[0]['lr']
                        })

                all_y_true_val = np.concatenate(all_y_true_val, axis=0)
                all_y_pred_val = np.concatenate(all_y_pred_val, axis=0)
                all_c_true_val = np.concatenate(all_c_true_val, axis=0)
                all_c_pred_val = np.concatenate(all_c_pred_val, axis=0)
                val_precision = precision_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_recall = recall_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_f1 = f1_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1), average='weighted')
                val_concept_accuracy = accuracy_score(all_c_true_val.argmax(1), all_c_pred_val.argmax(1))
                val_task_accuracy = accuracy_score(all_y_true_val.argmax(1), all_y_pred_val.argmax(1))

                scheduler.step(val_losses / len(val_loader))

                epoch_end_time = time.time()
                epoch_time = epoch_end_time - epoch_start_time

                print(f"Epoch {epoch+1}, Loss: {train_losses/len(train_loader)}, Train Accuracy: {train_correct/len(train_dataset)}, Val Loss: {val_losses/len(val_loader)}, Val Accuracy: {val_correct/len(val_dataset)}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}, Epoch Time: {epoch_time}")

                wandb.log({
                    'epoch': epoch + 1,
                    'loss': train_losses / len(train_loader),
                    'train_accuracy': train_correct / len(train_dataset),
                    'val_loss': val_losses / len(val_loader),
                    'val_accuracy': val_correct / len(val_dataset),
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'val_precision': val_precision,
                    'val_recall': val_recall,
                    'val_f1': val_f1,
                    'train_concept_accuracy': train_concept_accuracy,
                    'train_task_accuracy': train_task_accuracy,
                    'val_concept_accuracy': val_concept_accuracy,
                    'val_task_accuracy': val_task_accuracy,
                    'epoch_time': epoch_time
                })

                # Check for convergence
                if val_correct / len(val_dataset) >= target_accuracy and convergence_time is None:
                    convergence_time = time.time() - total_start_time
                    wandb.log({'convergence_time': convergence_time})
                    print(f"Convergence achieved at epoch {epoch+1} with validation accuracy {val_correct/len(val_dataset)}")

            total_training_time = time.time() - total_start_time
            wandb.log({'total_training_time': total_training_time})
            print(f"Total Training Time: {total_training_time}")

            print(f"\n Training on {dataset_name} using {model_name} has been completed!")
            torch.save(model, f'model_{model_name}_{dataset_name}.pth')
            torch.save(model.state_dict(), f'model_state_dict_{model_name}_{dataset_name}.pth')
            wandb.finish()

        print(f"===========================================================")


print(f"*********** ALL TRAINING ARE DONE - Check WandB ***********")


Training Started!
Training on DCRBase ... 
--------------------------------
The following dataset has been loaded successfully: XOR


-------------------------- Training XOR using DCRBase ----------------------
Epoch 1, Loss: 1.0362716669386083, Train Accuracy: 0.5021428571428571, Val Loss: 1.025920737873424, Val Accuracy: 0.5314285714285715, Train Precision: 0.7508929851510496, Train Recall: 0.5021428571428571, Train F1: 0.33963210026551766, Val Precision: 0.7608187633262261, Val Recall: 0.5314285714285715, Val F1: 0.4093071070569951, Epoch Time: 0.3013465404510498
Epoch 2, Loss: 1.0126415274359963, Train Accuracy: 0.5853571428571429, Val Loss: 0.9984437010504983, Val Accuracy: 0.5957142857142858, Train Precision: 0.6883322809848206, Train Recall: 0.5853571428571429, Train F1: 0.5206922476032667, Val Precision: 0.6242992594998098, Val Recall: 0.5957142857142858, Val F1: 0.5766772485872117, Epoch Time: 0.31109023094177246
Epoch 3, Loss: 0.9782215858047659, Train Accuracy: 0.6067857142857143, Val Loss: 0.9562058502977545, Val Accuracy: 0.6314285714285715, Train Precision: 0.6089654533949249, Train Recall: 0.6067857142

convergence_time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
epoch_time,▂▂▂▁▂▇▆▅▆▆▄▆▅▇█▄▂▁▁▁▃▁▂▂▂▁▂▃▁▁▂▂▁▂▁▁▂▄▁▂
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,███▇▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▂▂▃▄▄▄▄▄▆▆▇▇▇██████████████████████████
train_concept_accuracy,▁▂▄▇▇▇▇█████████████████████████████████
train_concept_loss,███▇▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1,▁▃▄▄▅▅▅▅▅▆▇▇▇███████████████████████████
train_precision,▄▂▁▂▃▄▄▄▄▅▆▇▇▇██████████████████████████


The following dataset has been loaded successfully: XNOR


-------------------------- Training XNOR using DCRBase ----------------------
Epoch 1, Loss: 1.0339123417030682, Train Accuracy: 0.49642857142857144, Val Loss: 1.023291371085427, Val Accuracy: 0.5314285714285715, Train Precision: 0.7502847438582154, Train Recall: 0.49642857142857144, Train F1: 0.33056105005115766, Val Precision: 0.7361263154031763, Val Recall: 0.5314285714285715, Val F1: 0.40041050284805774, Epoch Time: 0.28998541831970215
Epoch 2, Loss: 1.0096844475377689, Train Accuracy: 0.5603571428571429, Val Loss: 0.9937947175719521, Val Accuracy: 0.5842857142857143, Train Precision: 0.5840025510204082, Train Recall: 0.5603571428571429, Train F1: 0.5311294291902021, Val Precision: 0.5857245560561887, Val Recall: 0.5842857142857143, Val F1: 0.5827848423439831, Epoch Time: 0.2968778610229492
Epoch 3, Loss: 0.9720776053992185, Train Accuracy: 0.6678571428571428, Val Loss: 0.9485270001671531, Val Accuracy: 0.6928571428571428, Train Precision: 0.6856588956145333, Train Recall: 0.667857

...

convergence_time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
epoch_time,▂▃▁▃▂▁▁▁▂▁▁▁▁▁▃▁▁▂▁▁▂▂▂▂▂▂▅▆▅█▄▅▅▄▅▄▁▁▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▅▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▁▄▇▇▇▇▇▇▇▇▇▇███████████████████████████
train_concept_accuracy,▁▃▄▅▆▆▇▇████████████████████████████████
train_concept_loss,█▇▆▅▃▄▄▃▂▂▃▂▂▂▂▂▂▁▂▂▁▂▁▁▂▁▂▂▂▂▃▁▂▁▁▁▁▂▁▂
train_f1,▁▁▅▇▇▇█▇████████████████████████████████
train_precision,▁▁▆▇▇▇██████████████████████████████████


The following dataset has been loaded successfully: Dot


-------------------------- Training Dot using LLR3 ----------------------
Epoch 1, Loss: 1.0569172203540802, Train Accuracy: 0.5489285714285714, Val Loss: 1.0513964999805798, Val Accuracy: 0.5371428571428571, Train Precision: 0.5516040222892902, Train Recall: 0.5489285714285714, Train F1: 0.5267915455626404, Val Precision: 0.5674869831222138, Val Recall: 0.5371428571428571, Val F1: 0.4945535714285714, Epoch Time: 0.4829239845275879
Epoch 2, Loss: 1.0260874046520754, Train Accuracy: 0.5942857142857143, Val Loss: 1.015939398245378, Val Accuracy: 0.5828571428571429, Train Precision: 0.6221100300199979, Train Recall: 0.5942857142857143, Train F1: 0.5601181953129132, Val Precision: 0.6088142857142859, Val Recall: 0.5828571428571429, Val F1: 0.5629373897031548, Epoch Time: 0.39142680168151855
Epoch 3, Loss: 0.98495975272222, Train Accuracy: 0.6467857142857143, Val Loss: 0.9698037721894004, Val Accuracy: 0.6328571428571429, Train Precision: 0.6692026027802425, Train Recall: 0.6467857142857143

convergence_time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
epoch_time,█▄▄▁▃▂▁▃▂▂▁▂▁▁▂▂▁▂▂▁▂▂▁▂▄▁▂▂▁▁▂▂▇▅▇▇▇██▇
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,██▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▂▃▃▄▅▆▇▇▇▇▇▇▇██████████████████████████
train_concept_accuracy,▁▆▇▆▅▅▅▆▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇▇▇██▇██████████
train_concept_loss,██▆▇▆▄▄▃▃▃▃▃▄▃▃▃▂▃▃▃▂▃▃▃▂▂▂▃▂▃▃▃▃▃▂▂▁▂▂▁
train_f1,▁▂▃▄▅▅▆▇▇▇▇▇▇▇██████████████████████████
train_precision,▁▂▃▄▅▅▆▇▇▇▇▇▇▇██████████████████████████


*********** ALL TRAINING ARE DONE - Check WandB ***********
